In [25]:
import os
import numpy as np
import pandas as pd
import io

In [2]:
# Get every file into one list with the full filepaths
root_files = os.listdir('trees_all')
sub_files_to_flatten = [[f'{r}/{f}' for f in os.listdir(f'trees_all/{r}')] for r in root_files]
#Flatten list of files into one list
sub_files = [f'trees_all/{item}' for sublist in sub_files_to_flatten for item in sublist]

In [3]:
# Trying to look at file structure
# Total number of files
print(len(sub_files))
# Total number of files with the -noaa extension
print(len([sf for sf in sub_files if ('-noaa' in sf)]))
# Total number of files without the -noaa extension
print(len([sf for sf in sub_files if not ('-noaa' in sf)]))

9644
4775
4869


In [4]:
xt_set = {sf[:-9] for sf in sub_files if ('-noaa' in sf)}
normal_set = {sf[:-4] for sf in sub_files if not ('-noaa' in sf)}
# Number of files without matching sets of -noaa and non -noaa files
print(len(xt_set ^ normal_set))
# Files without matching sets
print(xt_set ^ normal_set)

110
{'trees_all/north_america/wi035', 'trees_all/north_america/wi022', 'trees_all/europe/finl086', 'trees_all/north_america/wi045', 'trees_all/north_america/wi059', 'trees_all/north_america/ia037', 'trees_all/north_america/tn', 'trees_all/asia/russ297', 'trees_all/north_america/ia036', 'trees_all/europe/finl083', 'trees_all/north_america/md009', 'trees_all/north_america/wi019', 'trees_all/europe/finl080', 'trees_all/europe/norw028', 'trees_all/north_america/wi028', 'trees_all/north_america/wi051', 'trees_all/europe/swed344', 'trees_all/north_america/wi024', 'trees_all/europe/norw024', 'trees_all/south_america/arge', 'trees_all/europe/norw027', 'trees_all/north_america/wi032', 'trees_all/europe/finl085', 'trees_all/europe/finl081', 'trees_all/north_america/wi057', 'trees_all/north_america/wi058', 'trees_all/europe/finl084', 'trees_all/north_america/wi034', 'trees_all/europe/brit', 'trees_all/europe/swed342', 'trees_all/europe/norw023', 'trees_all/north_america/wi021', 'trees_all/north_a

Seems for the most part, the files come in pairs and follow the format: {first 4 letters of country}{number}.rwl and {first 4 letters of country}{number}-noaa.rwl
- Files w/o -noaa ext: first 3 lines have info on the tree, + other metadata
- Files with -noaa ext: Many lines of text starting with "#" containing info about the data, then a line with column labels, the first of which is usually something with age

In [14]:
# Example of file with -noaa extension
# print(sub_files[0])
# file1 = open(sub_files[0],'r')
# print(file1.read())
# file1.close()

# Only keep noaa files
sub_files = [file for file in sub_files if 'noaa' in file]

In [ ]:
# Example of file without -noaa extension
print(sub_files[1])
file1 = open(sub_files[1],'r')
print(file1.read())
file1.close()

Hopefully not having matching sets isn't gonna be a problem if we just take all the data from the files with the -nooa extension and put it in one dataframe, and then take all the files with the extension and put that in a separate dataframe and find some way to merge them(maybe drop the ones without matching pairs?), but it would be good to know what that data actually is. 

In [16]:
# Attempt at cleaning a file w/ noaa xt
# Finds a line that starts with the word age and saves all the data
# below it (separated with newlines)
age_line = 0
with open(sub_files[0],'r') as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].startswith('age'):
            print(i)
            age_line = i
            break
    extracted = ' '.join(lines[age_line:])
    print(extracted)

74
age_CE	STAC32A_raw	STAC32B_raw	STAC34A_raw	STAC34B_raw	
  969 	NaN	NaN	203	126	
 970	NaN	NaN	191	186	
 971	NaN	NaN	140	138	
 972	NaN	NaN	172	174	
 973	NaN	NaN	221	221	
 974	NaN	NaN	155	159	
 975	NaN	NaN	204	209	
 976	NaN	NaN	131	136	
 977	NaN	NaN	153	154	
 978	NaN	NaN	169	173	
 979	NaN	NaN	161	161	
 980	NaN	NaN	203	206	
 981	NaN	NaN	143	141	
 982	NaN	NaN	166	174	
 983	NaN	NaN	189	194	
 984	NaN	NaN	170	167	
 985	NaN	NaN	190	197	
 986	NaN	NaN	189	198	
 987	NaN	NaN	173	196	
 988	NaN	NaN	156	157	
 989	NaN	NaN	249	237	
 990	NaN	NaN	153	158	
 991	NaN	NaN	248	246	
 992	NaN	NaN	248	242	
 993	NaN	NaN	215	223	
 994	NaN	NaN	212	223	
 995	119	138	171	181	
 996	152	144	145	150	
 997	148	145	164	163	
 998	189	191	188	181	
 999	298	292	299	298	
 1000	199	206	204	213	
 1001	219	217	230	238	
 1002	221	216	213	221	
 1003	225	228	299	272	
 1004	229	227	247	260	
 1005	201	189	214	207	
 1006	222	228	226	227	
 1007	200	186	206	197	
 1008	265	265	258	270	
 1009	191	204	194	200	
 1010	180	182	197	202	
 101

In [17]:
# Putting data from with a file w/ noaa extenstion in a dataframe 
data = io.StringIO(extracted)
df_test = pd.read_csv(data, sep="\t")
df_test.head()

,age_CE,STAC32A_raw,STAC32B_raw,STAC34A_raw,STAC34B_raw,Unnamed: 5
0,969,NaN,NaN,203,126,NaN
1,970,NaN,NaN,191,186,NaN
2,971,NaN,NaN,140,138,NaN
3,972,NaN,NaN,172,174,NaN
4,973,NaN,NaN,221,221,NaN


In [18]:
# Ignore but might be useful later
# Cleaning one without
with open(sub_files[1],'r') as f2:
    lines2 = f2.readlines()
d = [[w.strip() for w in textline.split(' ') if (w != '')] for textline in lines2[3:]]
# No idea what the columns should be
df2 = pd.DataFrame(d)
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,#,and,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,#,NOAA,Paleoclimatology,Program,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,#,National,Centers,for,Environmental,Information,(NCEI),None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,#---------------------------------------------...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,#,Template,Version,3.0,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [20]:
# Avg tree rings width for all non - NaN data per year
# mean() function doesn't count NaN values in the calculation
df_test['Avg'] = df_test.drop('age_CE',axis=1).mean(axis=1)
df_test.head()

,age_CE,STAC32A_raw,STAC32B_raw,STAC34A_raw,STAC34B_raw,Unnamed: 5,Avg
0,969,NaN,NaN,203,126,NaN,164.5
1,970,NaN,NaN,191,186,NaN,188.5
2,971,NaN,NaN,140,138,NaN,139.0
3,972,NaN,NaN,172,174,NaN,173.0
4,973,NaN,NaN,221,221,NaN,221.0


In [28]:
# Extracting info from metadata
file_name = sub_files[0]
north_lat = south_lat = west_lon = east_lon = 0
north_lat_ind = len('# Northernmost_Latitude:')
species = ''
earliest_year,latest_year = 0,0
organism_group = 0 #?
elevation = 0
with open(file_name,'r') as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].startswith('# Northernmost_Latitude:'):
            north_lat = float(lines[i].split(' ')[2])
        elif lines[i].startswith('# Southernmost_Latitude:'):
            south_lat = float(lines[i].split(' ')[2])
        elif lines[i].startswith('# Easternmost_Longitude:'):
            east_lon = float(lines[i].split(' ')[2])
        elif lines[i].startswith('# Westernmost_Longitude:'):
            west_lon = float(lines[i].split(' ')[2])
        elif lines[i].startswith('# Species_Name:'):
            species = lines[i].split(': ')[1]
        elif lines[i].startswith('# Earliest_Year:'):
            earliest_year = int(lines[i].split(': ')[1])
        elif lines[i].startswith('# Most_Recent_Year:'):
            latest_year = int(lines[i].split(': ')[1])
        elif lines[i].startswith('# Elevation:'):
            try:
                elevation = float(lines[i].split(': ')[1])
            except ValueError:
                elevation = np.nan
        
lat = (north_lat + south_lat)/2
lon = (east_lon + west_lon)/2

print(file_name)
print(organism_group)
print(lat)
print(lon)
print(species)
print(earliest_year)
print(latest_year)
print(elevation)

trees_all/europe/brit091-noaa.rwl
0
51.75041
-0.34223
Quercus spp. L.

969
1053
nan


In [21]:
cols = ['year','avg_tree_ring_width','lat','long','elevation','tree_species']
feats = [1,2,3,4,5,6]
df_final = pd.DataFrame(columns=cols)
#df_final.append(,ignore_index=True)

In [22]:
df_final = df_final.append({x:y for x in cols for y in feats},ignore_index=True)
df_final.head()

,year,avg_tree_ring_width,lat,long,elevation,tree_species
0,6,6,6,6,6,6
